##### 7.4 합성곱/폴링 계층 구현하기

##### 7.4.1 4차원 배열

- CNN의 계층 사이에 흐르는 데이터는 4차원 데이터
    - ex) (10, 1, 28, 28)  
    $\equiv$ 높이 28, 너비 28, 채널 1개인 데이터가 10개

In [2]:
import numpy as np

x = np.random.rand(10, 1, 28, 28)   # 무작위로 데이터 생성 
print(x.shape)

(10, 1, 28, 28)


In [3]:
print(x[0].shape)   # 첫 번째 데이터 접근
print(x[1].shape)   # 두 번째 데이터 접근

(1, 28, 28)
(1, 28, 28)


In [4]:
print(x[0, 0])  # 또는 x[0][0], 첫 번째 데이터의 첫 채널의 공간 데이터

[[7.03191987e-01 7.06253296e-01 1.54235595e-01 6.47425123e-01
  7.22746815e-01 2.05326319e-01 5.47133309e-01 6.79529232e-01
  9.87382394e-01 3.42760507e-01 1.85307994e-01 4.41650386e-01
  2.37367000e-01 2.91234537e-01 9.21417282e-01 5.34630913e-01
  2.27612365e-01 1.75353675e-01 4.54464675e-01 3.98766819e-03
  1.69408831e-01 2.75561692e-01 2.38106306e-01 2.59589561e-01
  3.30014902e-01 9.31422727e-02 3.81908292e-01 8.21757815e-01]
 [4.71203921e-03 4.94801009e-01 9.38207608e-01 1.55114888e-02
  8.05303607e-03 8.55424559e-01 9.07230056e-02 5.56661688e-01
  9.29349164e-01 4.04464157e-05 8.45448465e-01 2.06251488e-01
  9.04207577e-01 3.55250254e-01 9.99205018e-01 6.85630792e-01
  8.39261930e-01 5.38242813e-01 4.83819140e-01 2.20913516e-01
  9.90783353e-01 6.06283414e-01 7.19978273e-01 4.62240349e-01
  4.53367142e-01 3.89168518e-01 5.77705804e-01 7.12860466e-01]
 [9.87260372e-01 4.17918690e-01 5.29435483e-01 8.01134766e-01
  5.66255937e-01 4.47254609e-01 4.93724554e-01 8.15815936e-02
  9.93

##### 7.4.2 im2col로 데이터 전개

- 넘파이에서 원소 접근 시, for문을 사용하지 않는 것이 바람직  
- for문 대신 im2col 함수 사용해 CNN 구현  
- `im2col(image to column)': Cafe, Chainer등의 딥러닝 프레임워크에서 합성곱 계층 구현시 이용  
$\equiv$ 입력 데이터를 필터링(가중치 계산)하기 좋게 전개하는(펼치는) 함수

- 실제 상황에서는 스트라이드를 크게 잡지 않아,
$\to$ 필터의 적용 영역이 겹치는 경우가 많음  
$\to$ im2col로 전개한 후의 원소 수가 원래 블록의 원소 수보다 많아짐
$\to$ 평소보다 메모리 `소비가 많아지는 단점' 존재

- im2col로 입력 데이터를 2차원 데이터로 전개한 후  
$\to$ 합성곱 계층의 필터(가중치)를 1열로 전개하고  
$\to$ 두 행렬의 내적 연산

##### 7.4.3 합성곱 계층 구현

- im2col(input_data, filter_h, filter_w, stride=1, pad=0)  
    - input_data: (데이터 수, 채널 수, 높이, 너비)의 4차원 배열로 이뤄진 입력 데이터
    - filter_H, filter_w: 필터의 높이/너비
    - stride: 스트라이드
    - pad: 패딩

In [5]:
import sys, os
sys.path.append(os.pardir)
from util import im2col

x1 = np.random.rand(1, 3, 7, 7)     # (데이터 수 = 배치 크기, 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)    # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

# 75 == 필터의 원소 수(채널 3개, 5*5 데이터)
# 배치 크기 1일 때: (9, 75)
# 배치 크기 10일 때: (90. 75)

(9, 75)
(90, 75)


In [7]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1)  # 필터 전개
        out = np.dot(col, col_W) + self.b

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)    # 출력 데이터를 적절한 형상으로 변환

        return out

- 필터(가중치), 편향, 스트라이드, 패딩을 인수로 받아 초기화  
$\to$ 입력 데이터를 `im2col로 전개`  
$\to$ 필터를 `reshape`을 사용해 2차원 배열로 전개
$\to$ 전개한 두 행렬의 `내적` 연산

- `reshape`: 인자를 -1로 지정하면, 다차원 배열의 원소 수가 변환 후에도 똑같이 유지되도록 묶어줌  
$\equiv$ ex) (10, 3, 5, 5) 형상인 다차원 배열 W의 원소 수는 총 750개
$\to$ 해당 배열에 reshpae(10, -1) 호출 시  
$\to$ 750개의 원소를 10묶음인 (10, 75)인 배열로

- `tranpose`: 다차원 배열의 축 순서를 바꿔주는 함수  
$\equiv$ 인덱스 번호로 축의 순서 변경

- 역전파에서, im2col을 역으로 처리해야 함(col2im 사용)

##### 7.4.4 풀링 계층 구현

- 합성곱 계층과 동일하게 im2col 사용해 입력 데이터 전개  
- 채널이 독립적임을 주의  
$\to$ 전개한 행렬에서 행별 최댓값을 구하고
$\to$ 적절한 형상으로 성형

In [8]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        # 전개(1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)

        # 최댓값(2)
        out = np.max(col, axis=1)

        # 성형(3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

- `폴링 계층 구현`
    1. 입력 데이터 전개  
    2. 행별 최댓값 구하기: np.max 사용 가능, 인수 중 axis로 지정한 축마다 최댓값을 구할 수 있음
        - ex) np.max(x, axis=1)  
        $\equiv$ 입력 x의 1번째 차원의 축마다 최댓값 추출
    3. 적절한 모양으로 성형